In [1]:
import xmltodict

with open('ds_od_xml.xml') as f:
    d = xmltodict.parse(f.read())

In [2]:
from datetime import date, datetime

YMD_FORMAT = '%Y-%m-%d'

assert date(2022, 4, 29).strftime(YMD_FORMAT) == '2022-04-29'

In [3]:
def get_parsed_value(d, k):
    v = d[k]
    if v is None:
        return None
    if 'DATE' in k:
        return datetime.strptime(v, YMD_FORMAT)
    if 'AMOUNT' in k:
        return float(v)
    return v

In [4]:
from ds_types import Infraction
from typing import List, Union


def get_infractions(infraction_d_or_l: Union[List[dict], dict]) -> List[Infraction]:

    if isinstance(infraction_d_or_l, dict):
        infraction_d_l = [infraction_d_or_l]
    else:
        infraction_d_l = infraction_d_or_l

    infractions: List[Infraction] = []
    for infraction_d in infraction_d_l:
        infraction = Infraction(
            severity=get_parsed_value(infraction_d, 'SEVERITY'),
            deficiency=get_parsed_value(infraction_d, 'DEFICIENCY'),
            action=get_parsed_value(infraction_d, 'ACTION'),
            court_outcome=get_parsed_value(infraction_d, 'COURT_OUTCOME'),
            amount_fined=get_parsed_value(infraction_d, 'AMOUNT_FINED'),
        )
        infractions.append(infraction)

    return infractions


In [5]:
from ds_types import Establishment, Inspection, Infraction
from typing import List

for establishment_d in d['DINESAFE_DATA']['ESTABLISHMENT']:
    inspections: List[Inspection] = []
    for inspection_d in establishment_d['INSPECTION']:
        inspection = Inspection(
            status=get_parsed_value(inspection_d, 'STATUS'),
            date=get_parsed_value(inspection_d, 'DATE'),
            infraction=get_infractions(inspection_d.get('INFRACTION', []))
        )
        inspections.append(inspection)
    establishment = Establishment(
        id=get_parsed_value(establishment_d, 'ID'),
        name=get_parsed_value(establishment_d, 'NAME'),
        type=get_parsed_value(establishment_d, 'TYPE'),
        address=get_parsed_value(establishment_d, 'ADDRESS'),
        latitude=get_parsed_value(establishment_d, 'LATITUDE'),
        longitude=get_parsed_value(establishment_d, 'LONGITUDE'),
        status=get_parsed_value(establishment_d, 'STATUS'),
        inspection=inspections,
    )
    break
